# Мэтчинг товаров

**Задача:**

- разработать алгоритм, который для всех товаров из validation.csv предложит несколько вариантов наиболее похожих товаров из base; 
- оценить качество алгоритма по метрике accuracy@5. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial import distance
import faiss

In [2]:
base = pd.read_csv('data/base.csv')
train = pd.read_csv('data/train.csv')
validation_answer = pd.read_csv('data/validation_answer.csv')
validation = pd.read_csv('data/validation.csv')

In [3]:
display(base.info())
display(validation.info())
display(validation_answer.info())
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2918139 entries, 0 to 2918138
Data columns (total 73 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Id      object 
 1   0       float64
 2   1       float64
 3   2       float64
 4   3       float64
 5   4       float64
 6   5       float64
 7   6       float64
 8   7       float64
 9   8       float64
 10  9       float64
 11  10      float64
 12  11      float64
 13  12      float64
 14  13      float64
 15  14      float64
 16  15      float64
 17  16      float64
 18  17      float64
 19  18      float64
 20  19      float64
 21  20      float64
 22  21      float64
 23  22      float64
 24  23      float64
 25  24      float64
 26  25      float64
 27  26      float64
 28  27      float64
 29  28      float64
 30  29      float64
 31  30      float64
 32  31      float64
 33  32      float64
 34  33      float64
 35  34      float64
 36  35      float64
 37  36      float64
 38  37      float64
 39  38      float64
 40  

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 73 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Id      100000 non-null  object 
 1   0       100000 non-null  float64
 2   1       100000 non-null  float64
 3   2       100000 non-null  float64
 4   3       100000 non-null  float64
 5   4       100000 non-null  float64
 6   5       100000 non-null  float64
 7   6       100000 non-null  float64
 8   7       100000 non-null  float64
 9   8       100000 non-null  float64
 10  9       100000 non-null  float64
 11  10      100000 non-null  float64
 12  11      100000 non-null  float64
 13  12      100000 non-null  float64
 14  13      100000 non-null  float64
 15  14      100000 non-null  float64
 16  15      100000 non-null  float64
 17  16      100000 non-null  float64
 18  17      100000 non-null  float64
 19  18      100000 non-null  float64
 20  19      100000 non-null  float64
 21  20      100

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Id        100000 non-null  object
 1   Expected  100000 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 74 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Id      100000 non-null  object 
 1   0       100000 non-null  float64
 2   1       100000 non-null  float64
 3   2       100000 non-null  float64
 4   3       100000 non-null  float64
 5   4       100000 non-null  float64
 6   5       100000 non-null  float64
 7   6       100000 non-null  float64
 8   7       100000 non-null  float64
 9   8       100000 non-null  float64
 10  9       100000 non-null  float64
 11  10      100000 non-null  float64
 12  11      100000 non-null  float64
 13  12      100000 non-null  float64
 14  13      100000 non-null  float64
 15  14      100000 non-null  float64
 16  15      100000 non-null  float64
 17  16      100000 non-null  float64
 18  17      100000 non-null  float64
 19  18      100000 non-null  float64
 20  19      100000 non-null  float64
 21  20      100

In [4]:
base.head()

,Id,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
0,0-base,-115.083890,11.152912,-64.426760,-118.88089,216.482440,-104.698060,-469.070588,44.348083,120.915344,...,-42.808693,38.800827,-151.76218,-74.389090,63.66634,-4.703861,92.933610,115.269190,-112.756640,-60.830353
1,1-base,-34.562202,13.332763,-69.787610,-166.53348,57.680607,-86.098370,-85.076666,-35.637436,119.718636,...,-117.767525,41.100000,-157.82940,-94.446806,68.20211,24.346846,179.937930,116.834000,-84.888941,-59.524610
2,2-base,-54.233746,6.379371,-29.210136,-133.41383,150.895830,-99.435326,52.554795,62.381706,128.951450,...,-76.397800,46.011803,-207.14442,127.325570,65.56618,66.325680,81.073490,116.594154,-1074.464888,-32.527206
3,3-base,-87.520130,4.037884,-87.803030,-185.06763,76.369540,-58.985165,-383.182845,-33.611237,122.031910,...,-70.647940,-6.358921,-147.20105,-37.692750,66.20289,-20.566910,137.206940,117.474100,-1074.464888,-72.915490
4,4-base,-72.743850,6.522049,43.671265,-140.60803,5.820023,-112.074080,-397.711282,45.182500,122.167180,...,-57.199104,56.642403,-159.35184,85.944724,66.76632,-2.505783,65.315285,135.051590,-1074.464888,0.319401


Перед нами датасет base.csv с 2 918 139 записями, у каждой 72 столбца с данными и еще один столбец с id (в формате n-base, где n - порядковый норер записи)

In [5]:
validation.head()

,Id,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
0,100000-query,-57.372734,3.597752,-13.213642,-125.92679,110.74594,-81.279594,-461.003172,139.815720,112.880980,...,-75.51302,52.830902,-143.43945,59.051935,69.28224,61.927513,111.59253,115.140656,-1099.130485,-117.079360
1,100001-query,-53.758705,12.790300,-43.268543,-134.41762,114.44991,-90.520130,-759.626065,63.995087,127.117905,...,-79.44183,29.185436,-168.60590,-82.872443,70.76560,-65.975950,97.07716,123.391640,-744.442332,-25.009320
2,100002-query,-64.175095,-3.980927,-7.679249,-170.16093,96.44616,-62.377740,-759.626065,87.477554,131.270110,...,-134.79541,37.368730,-159.66231,-119.232725,67.71044,86.002060,137.63641,141.081630,-294.052271,-70.969604
3,100003-query,-99.286860,16.123936,9.837166,-148.06044,83.69708,-133.729720,58.576403,-19.046660,115.042404,...,-77.23611,44.100494,-132.53012,-106.318982,70.88396,23.577892,133.18396,143.252940,-799.363667,-89.392670
4,100004-query,-79.532920,-0.364173,-16.027431,-170.88495,165.45392,-28.291668,33.931936,34.411217,128.903980,...,-123.77025,45.635944,-134.25893,13.735359,70.61763,15.332115,154.56812,101.700640,-1171.892332,-125.307890


Перед нами датасет validation.csv с 100000 записями, у каждой 72 столбца с данными и еще один столбец с id (в формате n-query, где n - порядковый норер записи)

In [6]:
validation_answer.head()

,Id,Expected
0,100000-query,2676668-base
1,100001-query,91606-base
2,100002-query,472256-base
3,100003-query,3168654-base
4,100004-query,75484-base


В датасете validation_answer находятся ответы для validation.

In [7]:
train.head()

,Id,0,1,2,3,4,5,6,7,8,...,63,64,65,66,67,68,69,70,71,Target
0,0-query,-53.882748,17.971436,-42.117104,-183.93668,187.517490,-87.144930,-347.360606,38.307602,109.085560,...,70.107360,-155.80257,-101.965943,65.903790,34.457500,62.642094,134.763600,-415.750254,-25.958572,675816-base
1,1-query,-87.776370,6.806268,-32.054546,-177.26039,120.803330,-83.810590,-94.572749,-78.433090,124.915900,...,4.669178,-151.69771,-1.638704,68.170876,25.096191,89.974976,130.589630,-1035.092211,-51.276833,366656-base
2,2-query,-49.979565,3.841486,-116.118590,-180.40198,190.128430,-50.837620,26.943937,-30.447489,125.771164,...,78.039764,-169.14620,82.144186,66.008220,18.400496,212.409730,121.931470,-1074.464888,-22.547178,1447819-base
3,3-query,-47.810562,9.086598,-115.401695,-121.01136,94.652840,-109.255410,-775.150134,79.186520,124.003100,...,44.515266,-145.41675,93.990981,64.131350,106.061920,83.178760,118.277725,-1074.464888,-19.902788,1472602-base
4,4-query,-79.632126,14.442886,-58.903397,-147.05254,57.127068,-16.239529,-321.317964,45.984676,125.941284,...,45.028910,-196.09207,-117.626337,66.926220,42.456170,77.621765,92.479930,-1074.464888,-21.149351,717819-base


Перед нами датасет train.csv с 100000 записями. Это тренировочный датасет, в нем есть ид N-query и id товара из base.csv, который максимально похож на него (по мнению экспертов)

## Реализация без FAISS

Самая простая реализация -- это найти 5 ближайщих векторов. Но, к сожалению, нам потребуется слишком много времени: для каждого вектора из validation нужно пройтись по покаждому вектору из base. Это ориентироваочно 10<sup>5</sup> * 3 * 10<sup>6</sup> = 3 * 10<sup>11</sup> операций сравнений, что слишком много. Поэтому это нам не подходит.

Сделаем так, 
* разобъем (классифицируем) датасет base на 100-200 частей, 
* в каждой части посчитаем средний вектор,
* для каждого вектора из validation посчитаем какой из средних векторов ближе
* и уже для найденной группы векторов будем считать 5 минимальных расстояний

### Классификация датасета base

In [8]:
try:
    base = pd.read_csv('data/base_clustered.csv')
except:
    model = KMeans(n_clusters=200)

    data = base.drop('Id', axis=1)
    model.fit(data)
    all_predictions = model.predict(data)
    base['cluster'] = all_predictions
    base.to_csv('base_clustered.csv', index=False)


In [9]:
data = base.drop('Id', axis=1) 
base_vectors = []
for i in range(200):
    base_vectors.append(np.mean(data[data['cluster'] == i]))
    
base_vectors = pd.DataFrame(base_vectors).sort_values(by='cluster', ascending=True).drop('cluster', axis=1)
base_vectors

/Users/alexsavelyev/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3502: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,-87.483865,7.410844,-45.226707,-144.584674,110.911582,-73.814817,-97.135548,29.553578,122.619210,124.154542,...,-78.972521,37.160199,-149.818397,21.057203,67.881712,15.643578,72.117481,112.404793,-243.962998,-50.278991
1,-82.435949,7.956602,-43.924467,-146.275526,113.353112,-76.194968,-476.684165,19.158540,124.599684,118.276949,...,-76.489066,36.396099,-161.444346,-83.823142,67.659389,30.972251,79.057195,113.836984,-1074.899034,-51.320902
2,-84.110651,7.421150,-43.207856,-146.787523,111.456273,-76.044014,-65.937478,19.984497,124.096857,118.507821,...,-78.274715,37.014544,-159.297691,15.443641,67.668720,24.807993,76.206366,114.945825,-1084.239839,-49.682822
3,-93.743863,11.711394,-41.598750,-149.529545,107.661340,-58.361119,-506.658705,55.553003,124.740013,107.048375,...,-81.503309,-16.451386,-154.517113,14.751294,68.337992,11.647984,81.020147,120.273419,-75.719329,-56.373051
4,-85.066219,7.443608,-43.275807,-146.410949,110.536453,-74.936030,-707.000225,22.154570,123.811561,123.517647,...,-80.159462,36.508522,-155.441838,15.909049,67.727752,23.290440,75.666745,114.914289,-1086.357736,-48.878649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,-85.681900,7.720329,-52.337126,-145.684602,109.745574,-71.840752,-664.510425,16.327903,124.441523,130.091015,...,-77.409808,33.928543,-152.857029,9.630912,68.062281,33.808159,74.461141,115.941221,-994.456596,-40.875932
196,-85.029751,7.609359,-41.821194,-146.653068,111.176361,-74.336838,-505.167672,22.557034,123.584140,122.588103,...,-82.186326,35.748691,-154.240574,38.550061,67.817361,22.634790,73.940107,116.036179,-720.711400,-48.118185
197,-87.678824,8.394600,-52.382137,-143.727162,110.314530,-70.689198,-143.153804,23.972175,124.799445,126.545244,...,-76.282188,33.771859,-151.562852,8.770368,68.008265,29.289856,70.825017,116.329859,-1024.813080,-43.066081
198,-89.486273,7.404637,-47.438811,-142.737780,105.315192,-72.736325,-148.128263,19.362856,122.055626,118.585340,...,-78.945205,37.490485,-139.805165,-9.883731,67.808153,19.029194,73.131670,114.443322,-1031.629812,-47.925848


In [10]:
def get_nearest_neighbor(features, vector, top=1):
    distancies = [];
    for feature_index, row in features.iterrows():
        dis = distance.euclidean(np.array(features.loc[feature_index].values), vector)
        distancies.append(dis)
    distancies_df = pd.DataFrame(data=distancies, columns=['dist'], index=features.index)
    distancies_df = distancies_df.sort_values(by='dist', ascending=True) 
    
    return distancies_df.head(top).index.values

In [11]:
def calculate_accuracy5(predict, answer):
    points = 0
    for index in range(len(answer)):
        if answer[index] in predict[index]:
            points += 1
    return points/len(answer)

In [12]:
%%time
valid = validation.drop(['Id'], axis=1).head(100)
predicts = []
expected = []
for vector_index in range(len(valid)):
    # получаем index главного вектора
    result = get_nearest_neighbor(base_vectors, valid.loc[vector_index])
    features = data[data['cluster'] == result[0]].drop('cluster', axis=1)
    # получаем индексы топ 5 ближайщих соседей
    top_five_index = get_nearest_neighbor(features, valid.loc[vector_index], 5)
    # получаем 5 самых ближайщих товаров
    top_five = base.loc[top_five_index]['Id'].values
    
    predicts.append(top_five)
    expected.append(validation_answer.loc[vector_index]['Expected'])
    
calculate_accuracy5(predicts, expected)

CPU times: user 1min 43s, sys: 947 ms, total: 1min 44s
Wall time: 1min 44s


0.04

**Вывод:** высокой оценки добиться не удалось, а считается слишком долго.

## Реализация с FAISS

In [13]:
%%time
dim = 72
index = faiss.IndexFlatL2(dim)
index.add(data.drop('cluster', axis=1))
k = 5

predicts = []
expected = []
valid = validation.drop('Id', axis=1)
for vector_index in range(len(valid)):
    dist, ind = index.search(valid[vector_index:vector_index+1], k=k)
    top_five_index = ind[0]
    top_five = base.loc[top_five_index]['Id'].values
    
    predicts.append(top_five)
    expected.append(validation_answer.loc[vector_index]['Expected'])
    if vector_index % 100 == 0:
        print(vector_index)
    
calculate_accuracy5(predicts, expected)
    

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

0.13286

**Вывод:** Получилось улучшить оценку. Но не так высоко, как хотелось бы (13.28%)

## Возможные улучшения 
  - Поиграться с данными, попытаться проанализировать значения параметров
  - Попытаться найти разные алгоритмы разбиения на кластеры, проанализировать работу простых алгоритмов при разном кол-ве кластеров
  - Поиграться с параметрами faiss, попробовать разные расстояния
  - Попробовать Annoy